# ETL Pipeline

## Extract Data

In [1]:
# %pip install -q tweepy
# %pip install -q python-decouple
%pip install -U spacy
!python3 -m spacy download pt_core_news_sm 

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


     --------------------------------------- 13.0/13.0 MB 21.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')



[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# import necessay packages
import pandas as pd
import tweepy as tw

import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import RSLPStemmer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from functools import reduce
import operator
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import spacy


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\4YouSee\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\4YouSee\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\4YouSee\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\4YouSee\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [3]:
# config
consumer_key = 'ALS2SK37YuglyLYSyGGpfrbmY'
consumer_secret = '2Vyr3pBaHDsNaVX9IslmRHq25xR4zJjQjHpjUFTZzXj74rkvwS'
access_token = '1521183878576230400-4DzxTK7h5tI1gQ1f4QwoEWqfn0FNuV'
access_token_secret = 'sft4cbN41jMtVneJLD794rYRGJraHbb9ug4YELbZHFSyV'
bearer_token = 'AAAAAAAAAAAAAAAAAAAAAA6ScAEAAAAAnv1iKJAyg2INl1nA7%2BB0z%2FJbpK4%3DbrVth6B429DvCW0vsS4ufO6pLzmsAnhyYpaPRfWGoMWBKc6uRM'
user_id = '90676816'

In [4]:
# connect API
permission = tw.OAuthHandler(consumer_key, consumer_secret) # this object of ours will load the authorization request
permission.set_access_token(access_token, access_token_secret) # get ownership of our access object
client = tw.Client(bearer_token=bearer_token)
api = tw.API(permission) # use the API class, native to the library to generate the connection

In [5]:
query = 'Atletico-MG -filter:retweets OR Atlético Mineiro -filter:retweets OR #Galo -filter:retweets OR #ClubeAtléticoMineiro -filter:retweets'
galo_tweets = api.search_tweets(q=query, tweet_mode='extended', count=100)

id = []
created_at = []
full_text = []
favorite_count = []
retweet_count = []

for twitters in galo_tweets:
    id.append(twitters.id)
    created_at.append(twitters.created_at)
    full_text.append(twitters.full_text)
    favorite_count.append(twitters.favorite_count)
    retweet_count.append(twitters.retweet_count)

    # create dataframe
    df_final = {
      'tweet_id': id, 'created_at':created_at, 'text':full_text, 'likes':favorite_count, 
      'retweets':retweet_count
    }

In [6]:
search_tweets_df = pd.DataFrame(df_final)
search_tweets_df

,tweet_id,created_at,text,likes,retweets
0,1625096734614474752,2023-02-13 11:37:09+00:00,"Cruzeiro x Atlético-MG: veja onde assistir, es...",0,0
1,1625079843745419264,2023-02-13 10:30:01+00:00,Agenda de hoje - 13/02\nFutebol Brasileiro \n\...,0,0
2,1625075281395326977,2023-02-13 10:11:54+00:00,"Essa segunda-feira será em vão? Claro que não,...",1,0
3,1625071084625354752,2023-02-13 09:55:13+00:00,"Cruzeiro x Atlético-MG: veja onde assistir, es...",0,0
4,1625058589458182144,2023-02-13 09:05:34+00:00,"Cruzeiro x Atlético-MG: veja onde assistir, es...",23,0
...,...,...,...,...,...
95,1623476281214963713,2023-02-09 00:18:02+00:00,"Hulk brilha, Atlético-MG bate o Democrata e ma...",0,0
96,1623475795137110018,2023-02-09 00:16:06+00:00,6) Campeonato Mineiro na sua Claro tv+\n\n08/0...,2,0
97,1623475125289877505,2023-02-09 00:13:27+00:00,"Hulk faz dois golaços, Atlético-MG vence o Dem...",19,1
98,1623473962213572608,2023-02-09 00:08:49+00:00,"Hulk faz dois golaços, Atlético-MG vence o Dem...",24,0


In [7]:
# empty lists are created that will store the data extracted from the API call
id = []
date = []
text = []
retweets = []
answers = []
likes = []
mentions = []

# API call script with loop and pagination
for tweet in tw.Paginator(
    client.get_users_tweets, id=user_id, exclude=['retweets', 'replies'], #start_time = '2023-01-01T00:00:00Z',
    tweet_fields=['id', 'text', 'created_at', 'public_metrics'], max_results=100).flatten(limit=3200):

    # inserting the results in the lists
    id.append(tweet['id'])
    date.append(tweet['created_at'])
    text.append(tweet['text'])
    retweets.append(tweet['public_metrics']['retweet_count'])
    answers.append(tweet['public_metrics']['reply_count'])
    likes.append(tweet['public_metrics']['like_count'])
    mentions.append(tweet['public_metrics']['quote_count'])

    # create dataframe
    results = {
      'tweet_id':id, 'date':date, 'text':text, 'likes':likes, 
      'retweets':retweets, 'answers':answers, 'mentions':mentions
    }

In [8]:
paginator_df = pd.DataFrame(results)
paginator_df

,tweet_id,date,text,likes,retweets,answers,mentions
0,1625087385825382401,2023-02-13 11:00:00+00:00,"Hoje não posso, tem jogo do #Galo! 🖤🤍 https://...",1463,269,37,22
1,1624966842442829832,2023-02-13 03:01:00+00:00,💪🏾⚫⚪ HOJE TEM GAAAAAAAAAAAAAAAAAAAAAAAAALLLLLL...,3945,946,366,224
2,1624945199502819336,2023-02-13 01:35:00+00:00,"Dos gols de falta, qual o preferido do Homi? ⚽...",655,39,8,2
3,1624874735094968320,2023-02-12 20:55:00+00:00,⚽ Nono maior artilheiro da história do Atlétic...,1968,139,25,14
4,1624854602389815297,2023-02-12 19:35:00+00:00,🗞 Guia da partida! Confira todas as informaçõe...,628,40,10,4
...,...,...,...,...,...,...,...
825,1613248166564089858,2023-01-11 18:55:10+00:00,"🎙 Fala, Allan!\n\n📺 Veja, no YouTube, a entrev...",525,37,6,3
826,1613247301337874434,2023-01-11 18:51:43+00:00,🐔 Meio-campista Allan comenta o início de trab...,679,35,10,1
827,1613245805695238144,2023-01-11 18:45:47+00:00,🎥 #GALOTVAOVIVO! O meio-campista Allan concede...,300,20,1,1
828,1613200310066970624,2023-01-11 15:45:00+00:00,Priorize a sua proteção com o #MeuGaloAutoTruc...,318,16,5,0


In [9]:
test = pd.read_parquet('atleticomg-tweets_raw_own-twitters_atletico_own_tweets.parquet')
test

,tweet_id,date,text,likes,retweets,answers,mentions
0,1623486188043501570,2023-02-09 00:57:24+00:00,🎥 #GALOTVAOVIVO! Veja a entrevista do técnico ...,64,5,1,0
1,1623478987442749443,2023-02-09 00:28:48+00:00,"⚽ Tabela pela esquerda, assistência do Pantera...",533,66,9,2
2,1623474768681873408,2023-02-09 00:12:02+00:00,GANHAMO 🐔😎 https://t.co/VYXyEVQcag,4381,1374,117,70
3,1623473768570953729,2023-02-09 00:08:03+00:00,💪🏾⚫⚪ FIM DE JOGO! O ATLÉTICO GOLEIA O DEMOCRAT...,2184,524,152,99
4,1623473572898283522,2023-02-09 00:07:17+00:00,"Público: 21.216 pessoas\nRenda: R$ 507.430,70\...",831,60,16,4
...,...,...,...,...,...,...,...
1014,1605013867121319937,2022-12-20 01:35:00+00:00,🎥⚽️🎯🐔 https://t.co/UYZO8U3122,1578,97,18,12
1015,1604994687093260288,2022-12-20 00:18:47+00:00,🎥 #GALOTVAOVIVO: Nesta terça-feira tem coletiv...,480,32,6,1
1016,1604983668077785088,2022-12-19 23:35:00+00:00,💪🏽📸 Confira o álbum de fotos do treino de hoje...,893,54,5,2
1017,1604968569040683008,2022-12-19 22:35:00+00:00,🎥 Galo começa a calibrar a pontaria para 2023!...,910,63,16,8


In [10]:
df_clean = test.copy()

In [11]:
# Text clean (remove unnecessary characteres)
def clean_text(text):
    '''
    Function to put the texts in lowercase, remove the texts between
    square brackets, remove links, remove punctuation, and remove words that
    contain numbers

    :param text: (pandas series) 
    Respective column of a pandas dataframe that you want to clean
    '''
    # remove emojis
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text) # Remove the urls
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text) # Remove the punctuation
    text = re.sub('\n', '',text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub(emoj, '', text)
    return text

# Apply of "clean_text" function and add one more "cleaned" column in the original dataframe
df_clean['cleaned_text'] = df_clean['text'].apply(clean_text)

# Apply of nltk tokenize function and add one more "tokenized" column in the original dataframe
df_clean["tokenized_text"] = df_clean.apply(lambda row: nltk.word_tokenize(row["cleaned_text"], language='portuguese'), axis=1)

In [12]:
# Remove the stopwords
def remove_stopwords(text):
    '''
    Function that eliminates the stop words of a text

    :param text: (pandas series) 
    Respective column of a pandas dataframe that you want to remove the stopwords
    '''
    stop_words = set(stopwords.words("portuguese"))
    filtered_text = [word for word in text if word not in stop_words]
    return filtered_text

# Apply of "remove_stopwords" function and add one more column in the original dataframe
df_clean["nostopword_text"] = df_clean["tokenized_text"].apply(remove_stopwords)

In [13]:

nlp = spacy.load("pt_core_news_sm")


# Function to do the lemmatize task
def lemmatize_word(text):
    '''
    Function that lemmatize the words of a text

    :param text: (pandas series) 
    Respective column of a pandas dataframe that you want to lemmatize
    '''
    lemmas = [token.lemma_ for token in text]
    return lemmas

# Apply of "lemmatize_word" function and add one more "lemmatized" column in the original dataframe
df_clean["lemmatized_text"] = df_clean["nostopword_text"].apply(lemmatize_word)

AttributeError: 'str' object has no attribute 'lemma_'

In [ ]:
df_clean.head()

,tweet_id,date,text,likes,retweets,answers,mentions,cleaned_text,tokenized_text,nostopword_text,lemmatized_text
0,1623486188043501570,2023-02-09 00:57:24+00:00,🎥 #GALOTVAOVIVO! Veja a entrevista do técnico ...,64,5,1,0,galotvaovivo veja a entrevista do técnico edu...,"[galotvaovivo, veja, a, entrevista, do, técnic...","[galotvaovivo, veja, entrevista, técnico, edua...","[galotvaovivo, veja, entrevista, técnico, edua..."
1,1623478987442749443,2023-02-09 00:28:48+00:00,"⚽ Tabela pela esquerda, assistência do Pantera...",533,66,9,2,tabela pela esquerda assistência do pantera e...,"[tabela, pela, esquerda, assistência, do, pant...","[tabela, esquerda, assistência, pantera, pauli...","[tabela, esquerda, assistência, pantera, pauli..."
2,1623474768681873408,2023-02-09 00:12:02+00:00,GANHAMO 🐔😎 https://t.co/VYXyEVQcag,4381,1374,117,70,ganhamo,[ganhamo],[ganhamo],[ganhamo]
3,1623473768570953729,2023-02-09 00:08:03+00:00,💪🏾⚫⚪ FIM DE JOGO! O ATLÉTICO GOLEIA O DEMOCRAT...,2184,524,152,99,fim de jogo o atlético goleia o democratasl ...,"[fim, de, jogo, o, atlético, goleia, o, democr...","[fim, jogo, atlético, goleia, democratasl, faz...","[fim, jogo, atlético, goleia, democratasl, faz..."
4,1623473572898283522,2023-02-09 00:07:17+00:00,"Público: 21.216 pessoas\nRenda: R$ 507.430,70\...",831,60,16,4,público pessoasrenda r camxdfc,"[público, pessoasrenda, r, camxdfc]","[público, pessoasrenda, r, camxdfc]","[público, pessoasrenda, r, camxdfc]"
5,1623472423042535427,2023-02-09 00:02:42+00:00,💣 Potência do chute: 99\n🎯 Precisão do chute: ...,1097,173,17,29,potência do chute precisão do chute,"[potência, do, chute, precisão, do, chute]","[potência, chute, precisão, chute]","[potência, chute, precisão, chute]"
6,1623471066403176451,2023-02-08 23:57:19+00:00,Assista à jogada! #VamoGalo 🏴🏳️ https://t.co/s...,172,6,0,0,assista à jogada vamogalo,"[assista, à, jogada, vamogalo]","[assista, jogada, vamogalo]","[assista, jogada, vamogalo]"
7,1623470870835482624,2023-02-08 23:56:32+00:00,"⏱️ 36‘- UUUUUUUUHHHH! Falta do lado esquerdo, ...",251,7,4,1,⏱ ‘ uuuuuuuuhhhh falta do lado esquerdo hyoran...,"[⏱, ‘, uuuuuuuuhhhh, falta, do, lado, esquerdo...","[⏱, ‘, uuuuuuuuhhhh, falta, lado, esquerdo, hy...","[⏱, ‘, uuuuuuuuhhhh, falta, lado, esquerdo, hy..."
8,1623470599979802625,2023-02-08 23:55:28+00:00,Hulk à la Didi Folha Seca! Que classe... 🍃🎯 ht...,816,108,9,13,hulk à la didi folha seca que classe,"[hulk, à, la, didi, folha, seca, que, classe]","[hulk, la, didi, folha, seca, classe]","[hulk, la, didi, folha, seca, classe]"
9,1623469492910059520,2023-02-08 23:51:04+00:00,⏱️ 31‘- Substituição no Atlético:\n\n🔺 Ademir\...,187,7,29,10,⏱ ‘ substituição no atlético ademir allanvamog...,"[⏱, ‘, substituição, no, atlético, ademir, all...","[⏱, ‘, substituição, atlético, ademir, allanva...","[⏱, ‘, substituição, atlético, ademir, allanva..."


In [ ]:
pd.set_option('display.max_rows', None)